In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

load_dotenv(override=True)

True

In [4]:
#for checking sendgrid email

def send_email_test():
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("henrychu286@gmail.com")
    to_email = To("henrychu286@gmail.com")
    content = Content("text/plain", "This is SendGrid test email")
    mail = Mail(from_email, to_email, "This is SendGrid test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_email_test()

202


In [8]:
professional_agent_instructions = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

engaging_agent_instructions = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

busy_agent_instructions = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [21]:
professional_sale_agent = Agent(
    name="Professional Sales Agent",
    instructions=professional_agent_instructions,
    model="gpt-4o-mini"
)

engaging_sale_agent = Agent(
    name="Engaging Sales Agent",
    instructions=engaging_agent_instructions,
    model="gpt-4o-mini"
)

busy_sale_agent = Agent(
    name="Busy Sales Agent",
    instructions=busy_agent_instructions,
    model="gpt-4o-mini"
)

In [ ]:
result = Runner.run_streamed(professional_sale_agent, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data, end="", flush=True)


ResponseTextDeltaEvent(content_index=0, delta='Subject', item_id='msg_04f3c1588029ffc300696cf7445ec48190bb6e5fee73b641e2', logprobs=[], output_index=0, sequence_number=4, type='response.output_text.delta', obfuscation='XazGl0c6B')ResponseTextDeltaEvent(content_index=0, delta=':', item_id='msg_04f3c1588029ffc300696cf7445ec48190bb6e5fee73b641e2', logprobs=[], output_index=0, sequence_number=5, type='response.output_text.delta', obfuscation='JIF8GRjTHYMmDCa')ResponseTextDeltaEvent(content_index=0, delta=' Stream', item_id='msg_04f3c1588029ffc300696cf7445ec48190bb6e5fee73b641e2', logprobs=[], output_index=0, sequence_number=6, type='response.output_text.delta', obfuscation='dqkLxvuW2')ResponseTextDeltaEvent(content_index=0, delta='line', item_id='msg_04f3c1588029ffc300696cf7445ec48190bb6e5fee73b641e2', logprobs=[], output_index=0, sequence_number=7, type='response.output_text.delta', obfuscation='zA0dZ1WRgxNn')ResponseTextDeltaEvent(content_index=0, delta=' Your', item_id='msg_04f3c1588029

In [ ]:
import asyncio


system_prompt = "Write a cold sales mail"

with trace("Parallel cold mails"):
    results = await asyncio.gather(
        Runner.run(professional_sales_agent, system_prompt),
        Runner.run(engaging_sale_agent, system_prompt),
        Runner.run(busy_sale_agent)
    )